In [80]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation as cv
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [81]:
#不要なPassengerId,Cabinを削除。カテゴリー変数を数値に。Embarkedの穴を埋める。目的変数を取り出す。
df = pd.read_csv("train.csv")
y = df["Survived"].as_matrix()
df = df.drop(["PassengerId","Cabin"], axis = 1)
df = df.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
df = df.fillna(value = {'Embarked': 1})

#敬称ごとに変数を追加
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i and "Mrs" not in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append(4)
df['honor'] = np.array(honor)

#家族の人数を追加
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

#年齢の欠損を敬称別の中央値で埋める。
df = df.fillna(value = {"Age":-1})
a = df.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df[df["honor"]==i[1]]["Age"].median()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df = df.drop(["Age","honor"], axis = 1)
df = pd.concat([df, frame], axis=1)

In [82]:
#家族別の生存率を追加
name_df_train = df.iloc[:,2]
survive_array = df.iloc[:,0]

name_df = name_df_train

name_array = []
name_count = []
name_survive_count = []
name_survive_ratio = []

name_i = 0

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    if name_split_first in name_array:
        name_count[name_array.index(name_split_first)] = name_count[name_array.index(name_split_first)] + 1
        name_survive_count[name_array.index(name_split_first)] = name_survive_count[name_array.index(name_split_first)] + survive_array[name_i]                            
    else:
        name_array.append(name_split_first)
        name_count.append(1)
        name_survive_count.append(survive_array[name_i])
    name_i = name_i + 1
        
count = 0
for i in name_count:
    if name_count[count] == 1:
        name_survive_ratio.append(0)
    else:
        name_survive_ratio.append(name_survive_count[count]/(name_count[count]))
    count = count + 1
    
survive_ratio_array = []    

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    survive_ratio_array.append(name_survive_ratio[name_array.index(name_split_first)])

df['familysurviveratio'] = np.array(survive_ratio_array)


In [83]:
df.head()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
0,0,3,"Braund, Mr. Owen Harris",0,1,0,A/5 21171,7.2500,0.0,2,22.0,1.0,0.0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,PC 17599,71.2833,1.0,2,38.0,3.0,0.0
2,1,3,"Heikkinen, Miss. Laina",1,0,0,STON/O2. 3101282,7.9250,0.0,1,26.0,2.0,0.0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,0,113803,53.1000,0.0,2,35.0,3.0,0.5
4,0,3,"Allen, Mr. William Henry",0,0,0,373450,8.0500,0.0,1,35.0,1.0,0.5


In [84]:
df.corr()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
Survived,1.000000,-0.338481,0.543351,-0.035322,0.081629,0.257307,0.111203,0.016639,-0.057299,0.399261,0.411608
Pclass,-0.338481,1.000000,-0.131900,0.083081,0.018443,-0.549500,0.040138,0.065997,-0.371948,-0.246128,-0.213213
Sex,0.543351,-0.131900,1.000000,0.114631,0.245489,0.182333,0.121266,0.200988,-0.082709,0.758148,0.216081
SibSp,-0.035322,0.083081,0.114631,1.000000,0.414838,0.159651,-0.061591,0.890712,-0.258254,-0.014667,0.166872
Parch,0.081629,0.018443,0.245489,0.414838,1.000000,0.216225,-0.080277,0.783111,-0.166455,0.126376,0.289480
Fare,0.257307,-0.549500,0.182333,0.159651,0.216225,1.000000,0.065492,0.217138,0.111774,0.145413,0.194766
Embarked,0.111203,0.040138,0.121266,-0.061591,-0.080277,0.065492,1.000000,-0.082200,-0.059860,0.063359,0.042770
FamilySize,0.016639,0.065997,0.200988,0.890712,0.783111,0.217138,-0.082200,1.000000,-0.259666,0.053111,0.258671
Age,-0.057299,-0.371948,-0.082709,-0.258254,-0.166455,0.111774,-0.059860,-0.259666,1.000000,0.224600,-0.087234
honor,0.399261,-0.246128,0.758148,-0.014667,0.126376,0.145413,0.063359,0.053111,0.224600,1.000000,0.163514


In [42]:
df = df.drop(["Name","Ticket"],axis=1)


In [43]:
df = df.drop(["Survived","SibSp","Parch"],axis=1)


In [44]:
df.head()

,Pclass,Sex,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
0,3,0,7.2500,0.0,2,22.0,1.0,0.0
1,1,1,71.2833,1.0,2,38.0,3.0,0.0
2,3,1,7.9250,0.0,1,26.0,2.0,0.0
3,1,1,53.1000,0.0,2,35.0,3.0,0.5
4,3,0,8.0500,0.0,1,35.0,1.0,0.5


In [45]:
X = df.iloc[:,:].values

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [47]:
mod = xgb.XGBClassifier()
mod.fit(X_train, y_train)
print('train accuracy: %.3f' % mod.score(X_train, y_train))
print('test  accuracy: %.3f' % mod.score(X_test, y_test))

train accuracy: 0.902
test  accuracy: 0.869


In [48]:
kf = cv.KFold(n=len(X_train), n_folds=5, shuffle=True)
param_grid = {'learning_rate':[0.1],'n_estimators':[1000],'max_depth':[3,5],'min_child_weight':[1,2,3],'max_delta_step':[5],
'gamma':[0,3,10],'subsample':[0.8],'colsample_bytree':[0.8],'objective':['binary:logistic'],'nthread':[4],'scale_pos_weight':[1],
'seed':[0]}
clf = GridSearchCV(XGBClassifier(), param_grid=param_grid, cv=kf, scoring='roc_auc')

In [49]:
clf.fit(X_train, y_train)


GridSearchCV(cv=sklearn.cross_validation.KFold(n=623, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.1], 'n_estimators': [1000], 'max_depth': [3, 5], 'min_child_weight': [1, 2, 3], 'max_delta_step': [5], 'gamma': [0, 3, 10], 'subsample': [0.8], 'colsample_bytree': [0.8], 'objective': ['binary:logistic'], 'nthread': [4], 'scale_pos_weight': [1], 'seed': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [50]:
clf.score(X_test, y_test)
print("Best parameters: %s" % clf.best_params_)
print("Best auroc score: %s" % clf.best_score_)

Best parameters: {'colsample_bytree': 0.8, 'gamma': 3, 'learning_rate': 0.1, 'max_delta_step': 5, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 1000, 'nthread': 4, 'objective': 'binary:logistic', 'scale_pos_weight': 1, 'seed': 0, 'subsample': 0.8}
Best auroc score: 0.892450331617


In [51]:
mod2 = xgb.XGBClassifier(colsample_bytree = 0.8, gamma = 3, learning_rate = 0.1, max_delta_step = 5, 
max_depth = 5, min_child_weight = 1, n_estimators = 1000, nthread = 4, objective = 'binary:logistic', 
scale_pos_weight = 1, seed = 0,  subsample = 0.8)
mod2.fit(X_train, y_train)
print('train accuracy: %.3f' % mod2.score(X_train, y_train))
print('test  accuracy: %.3f' % mod2.score(X_test, y_test))

train accuracy: 0.915
test  accuracy: 0.873


In [63]:
#テストデータにも同様の処理をする
df2 = pd.read_csv("test.csv")
df2 = df2.drop(["PassengerId","Cabin"], axis = 1)
df2 = df2.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
df2 = df2.fillna(value = {'Embarked': 1})

#敬称ごとに変数を追加
honor1 = []
for i in df2["Name"]:
    if "Master" in i :
        honor1.append(0)
    elif "Mr" in i and "Mrs" not in i :
        honor1.append(1)
    elif "Miss" in i :
        honor1.append(2)
    elif "Mrs" in i :
        honor1.append(3)
    else:
        honor1.append(4)
df2['honor'] = np.array(honor1)

#家族の人数を追加
df2["FamilySize"] = df2["SibSp"] + df2["Parch"] + 1

#年齢の欠損を敬称別の中央値で埋める。
df2 = df2.fillna(value = {"Age":-1})
a = df2.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df2[df2["honor"]==i[1]]["Age"].median()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df2 = df2.drop(["Age","honor"], axis = 1)
df2 = pd.concat([df2, frame], axis=1)
df2[:10]

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,FamilySize,Age,honor
0,3,"Kelly, Mr. James",0,0,0,330911,7.8292,2,1,34.5,1.0
1,3,"Wilkes, Mrs. James (Ellen Needs)",1,1,0,363272,7.0000,0,2,47.0,3.0
2,2,"Myles, Mr. Thomas Francis",0,0,0,240276,9.6875,2,1,62.0,1.0
3,3,"Wirz, Mr. Albert",0,0,0,315154,8.6625,0,1,27.0,1.0
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1,1,3101298,12.2875,0,3,22.0,3.0
5,3,"Svensson, Mr. Johan Cervin",0,0,0,7538,9.2250,0,1,14.0,1.0
6,3,"Connolly, Miss. Kate",1,0,0,330972,7.6292,2,1,30.0,2.0
7,2,"Caldwell, Mr. Albert Francis",0,1,1,248738,29.0000,0,3,26.0,1.0
8,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",1,0,0,2657,7.2292,1,1,18.0,3.0
9,3,"Davies, Mr. John Samuel",0,2,0,A/4 48871,24.1500,0,3,21.0,1.0


In [64]:
name_df_test = df2.iloc[:,1]
survive_ratio_test =[]
for name in name_df_test:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')                                         
    if name_split_first in name_array:
        survive_ratio_test.append(name_survive_ratio[name_array.index(name_split_first)])
    else:
        survive_ratio_test.append(0)
df2= df2.drop(["Name","Ticket"], axis = 1)        
df2['familysurviveratio'] = np.array(survive_ratio_test)


In [65]:
name_survive_ratio[:5]

[0.0, 0, 0, 0.5, 0.5]

In [66]:
name_array[:5]

['Braund', 'Cumings', 'Heikkinen', 'Futrelle', 'Allen']

In [67]:
survive_ratio_test[:5]

[0.75, 0, 0, 0, 0]

In [68]:
df2.count()

Pclass                418
Sex                   418
SibSp                 418
Parch                 418
Fare                  417
Embarked              418
FamilySize            418
Age                   418
honor                 418
familysurviveratio    418
dtype: int64

In [75]:
df2.fillna(df2.mean())
df2 = df2.drop(["SibSp","Parch"],axis=1)

In [76]:
x = df2.iloc[:,:].values

In [77]:
y1= mod2.predict(x)

In [78]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y1})
dfnew2.to_csv("pred36.csv", index = False)